# Question: How do legal status and gender influence involvement in bribery and corruption across European countries?

In [187]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import ipywidgets as widgets
from IPython.display import display
from joypy import joyplot
from es_ds_loader import *
import joypy
import matplotlib.pyplot as plt

In [188]:
loader = EurostatDataLoader(cache_expiry=1800)
df = loader.load_dataset('crim_just_bri')

(('format', 'json'),)
('crim_just_bri', frozenset({('format', 'json')}))


In [189]:
"""#print(df)
df = df.dropna()
df = df[['geo', 'time', 'sex', 'leg_stat', 'unit', 'value']]
df = df[df['sex'] == 'Total']
print(df)""" # chart 1 (heatmap)

"#print(df)\ndf = df.dropna()\ndf = df[['geo', 'time', 'sex', 'leg_stat', 'unit', 'value']]\ndf = df[df['sex'] == 'Total']\nprint(df)"

Plot 1: Crime trends by age group over time
Joyplot: x-axis = year, y-axis = age groups, parameter = countries



In [190]:
""" 

# 1. Dein DataFrame vorbereiten # chart 2 (piechart)
df = df.dropna()
df = df[['geo', 'time', 'sex', 'leg_stat', 'unit', 'value']]

# 2. Zeilen ohne 'Total' im Geschlecht herausfiltern
filtered_df = df[df['sex'] = 'Total']

# 3. Gruppierung nach 'geo', 'time', 'leg_stat', 'sex' und 'unit', Berechnung der Summe für 'value'
grouped_df = filtered_df.groupby(['geo', 'time', 'leg_stat', 'sex', 'unit'])['value'].sum().reset_index()

# 4. Berechnung der Gesamtzahl pro 'geo', 'time', 'leg_stat' (unter Berücksichtigung der 'unit')
total_for_leg_stat = grouped_df.groupby(['geo', 'time', 'leg_stat', 'unit'])['value'].transform('sum')

# 5. Berechnung des Prozentsatzes und Hinzufügen der Spalte
grouped_df['percentage'] = (grouped_df['value'] / total_for_leg_stat) * 100

# 6. Ergebnis anzeigen
print(grouped_df)
"""

" \n\n# 1. Dein DataFrame vorbereiten # chart 2 (piechart)\ndf = df.dropna()\ndf = df[['geo', 'time', 'sex', 'leg_stat', 'unit', 'value']]\n\n# 2. Zeilen ohne 'Total' im Geschlecht herausfiltern\nfiltered_df = df[df['sex'] = 'Total']\n\n# 3. Gruppierung nach 'geo', 'time', 'leg_stat', 'sex' und 'unit', Berechnung der Summe für 'value'\ngrouped_df = filtered_df.groupby(['geo', 'time', 'leg_stat', 'sex', 'unit'])['value'].sum().reset_index()\n\n# 4. Berechnung der Gesamtzahl pro 'geo', 'time', 'leg_stat' (unter Berücksichtigung der 'unit')\ntotal_for_leg_stat = grouped_df.groupby(['geo', 'time', 'leg_stat', 'unit'])['value'].transform('sum')\n\n# 5. Berechnung des Prozentsatzes und Hinzufügen der Spalte\ngrouped_df['percentage'] = (grouped_df['value'] / total_for_leg_stat) * 100\n\n# 6. Ergebnis anzeigen\nprint(grouped_df)\n"

In [191]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Output, Input

# Beispiel-Daten (laden deinen Datensatz)
# df = pd.read_csv("dein_datensatz.csv")

# Filter-Optionen erstellen
countries = sorted(df['geo'].unique())
years = sorted(df['time'].unique())
units = sorted(df['unit'].unique())

# Dash App Setup
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Nested Pie Chart - Legal Status & Sex", style={'textAlign': 'center'}),

    # Dropdowns für Filter
    html.Div([
        html.Label('Land:'),
        dcc.Dropdown(
            options=[{'label': c, 'value': c} for c in countries],
            value=countries[0],
            id='country-dropdown'
        ),
    ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),

    html.Div([
        html.Label('Jahr:'),
        dcc.Dropdown(
            options=[{'label': y, 'value': y} for y in years],
            value=years[0],
            id='year-dropdown'
        ),
    ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),

    html.Div([
        html.Label('Unit:'),
        dcc.Dropdown(
            options=[{'label': u, 'value': u} for u in units],
            value=units[0],
            id='unit-dropdown'
        ),
    ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),

    # Graph für das Nested Pie Chart
    dcc.Graph(id='nested-pie-chart'),
    
    # Tabelle, die die Prozentsätze anzeigt
    html.Div(id='table-output')
])


# Callback für das Nested Pie Chart und die Tabelle mit Prozentwerten
@app.callback(
    [Output('nested-pie-chart', 'figure'),
     Output('table-output', 'children')],
    Input('country-dropdown', 'value'),
    Input('year-dropdown', 'value'),
    Input('unit-dropdown', 'value')
)
def update_pie(selected_country, selected_year, selected_unit):
    # Filtern des DataFrames basierend auf den Dropdown-Auswahlen
    filtered_df = df[
        (df['geo'] == selected_country) &
        (df['time'] == selected_year) &
        (df['unit'] == selected_unit)
    ]
    
    # **Zeilen ohne 'Total' im Geschlecht herausfiltern**
    filtered_df = filtered_df[filtered_df['sex'] != 'Total']

    # Gruppierung nach Legal Status und Sex und Berechnung der Gesamtzahl für Legal Status
    pie_data = filtered_df.groupby(['leg_stat', 'sex'])['value'].sum().reset_index()

    # Berechnung der Gesamtzahl für den Legal Status
    total_per_leg_stat = pie_data.groupby('leg_stat')['value'].transform('sum')

    # Berechnung des Prozentsatzes
    pie_data['percentage'] = (pie_data['value'] / total_per_leg_stat) * 100
    print(pie_data)

    # Erstellen des Nested Pie Charts
    fig = px.sunburst(
        pie_data,
        path=['leg_stat', 'sex'],  # Hierarchie: Legal Status -> Sex
        values='value',  # Größe der Segmente
        title=f"Nested Pie Chart für {selected_country} ({selected_year}) - {selected_unit}",
        hover_data=["percentage"]  # Zeige den Prozentsatz im Hover
    )

    # Generiere eine Tabelle, die die Prozentsätze anzeigt
    table_data = pie_data[['leg_stat', 'sex', 'value', 'percentage']]

    table_html = html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in table_data.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(row['leg_stat']),
                html.Td(row['sex']),
                html.Td(row['value']),
                html.Td(f"{row['percentage']:.2f}%")
            ]) for _, row in table_data.iterrows()
        ])
    ])

    return fig, table_html


# App starten
if __name__ == '__main__':
    app.run_server(debug=True)


           leg_stat      sex  value  percentage
0  Convicted person  Females   20.0   16.129032
1  Convicted person    Males  104.0   83.870968
2  Suspected person  Females   38.0   10.614525
3  Suspected person    Males  320.0   89.385475


In [192]:
print(pie_data)

           leg_stat      sex     value  percentage
0  Convicted person  Females   2822.88   11.353944
1  Convicted person    Males  22039.67   88.646056
2  Suspected person  Females  11741.54   13.564436
3  Suspected person    Males  74819.67   86.435564


In [193]:
"""import pandas as pd
import plotly.express as px
import pycountry
from dash import Dash, dcc, html, Output, Input

# 1. CSV laden


# 2. Länder-Korrektur
country_corrections = {
    "Türkiye": "Turkey",
    "Czechia": "Czech Republic",
    "Kosovo*": "Kosovo",
    "North Macedonia": "Macedonia",
    "United Kingdom": "United Kingdom",
    "Russian Federation": "Russia",
    "Bosnia and Herzegovina": "Bosnia and Herzegovina"
}

def get_iso3_fixed(country_name):
    name = country_corrections.get(country_name, country_name)
    try:
        return pycountry.countries.lookup(name).alpha_3
    except:
        return None

df = df[df['sex'] == 'Total']
df['iso_alpha'] = df['geo'].apply(get_iso3_fixed)
df = df.dropna(subset=['iso_alpha'])

# 3. Unique Filter-Optionen
years = sorted(df['time'].unique())
leg_stats = sorted(df['leg_stat'].unique())
units = sorted(df['unit'].unique())

# 4. Dash App Setup
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Europa Heatmap - Legal Status Analyse", style={'textAlign': 'center'}),

    html.Div([
        html.Label('Jahr:'),
        dcc.Dropdown(
            options=[{'label': y, 'value': y} for y in years],
            value=years[0],
            id='year-dropdown'
        ),
    ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),

    html.Div([
        html.Label('Legal Status:'),
        dcc.Dropdown(
            options=[{'label': l, 'value': l} for l in leg_stats],
            value=leg_stats[0],
            id='legstat-dropdown'
        ),
    ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),

    html.Div([
        html.Label('Unit:'),
        dcc.Dropdown(
            options=[{'label': u, 'value': u} for u in units],
            value=units[0],
            id='unit-dropdown'
        ),
    ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),

    dcc.Graph(id='heatmap')
])


# 5. Callback für Interaktivität
@app.callback(
    Output('heatmap', 'figure'),
    Input('year-dropdown', 'value'),
    Input('legstat-dropdown', 'value'),
    Input('unit-dropdown', 'value')
)
def update_map(selected_year, selected_legstat, selected_unit):
    filtered_df = df[
        (df['time'] == selected_year) &
        (df['leg_stat'] == selected_legstat) &
        (df['unit'] == selected_unit)
    ]

    fig = px.choropleth(
        filtered_df,
        locations="iso_alpha",
        color="value",
        hover_name="geo",
        color_continuous_scale="Reds",
        scope="europe",
        title=f"Heatmap {selected_legstat} ({selected_year}) - Unit: {selected_unit}"
    )

    fig.update_geos(showcountries=True, showcoastlines=True, showland=True, fitbounds="locations")
    return fig


if __name__ == '__main__':
    app.run_server(debug=True)
"""

'import pandas as pd\nimport plotly.express as px\nimport pycountry\nfrom dash import Dash, dcc, html, Output, Input\n\n# 1. CSV laden\n\n\n# 2. Länder-Korrektur\ncountry_corrections = {\n    "Türkiye": "Turkey",\n    "Czechia": "Czech Republic",\n    "Kosovo*": "Kosovo",\n    "North Macedonia": "Macedonia",\n    "United Kingdom": "United Kingdom",\n    "Russian Federation": "Russia",\n    "Bosnia and Herzegovina": "Bosnia and Herzegovina"\n}\n\ndef get_iso3_fixed(country_name):\n    name = country_corrections.get(country_name, country_name)\n    try:\n        return pycountry.countries.lookup(name).alpha_3\n    except:\n        return None\n\ndf = df[df[\'sex\'] == \'Total\']\ndf[\'iso_alpha\'] = df[\'geo\'].apply(get_iso3_fixed)\ndf = df.dropna(subset=[\'iso_alpha\'])\n\n# 3. Unique Filter-Optionen\nyears = sorted(df[\'time\'].unique())\nleg_stats = sorted(df[\'leg_stat\'].unique())\nunits = sorted(df[\'unit\'].unique())\n\n# 4. Dash App Setup\napp = Dash(__name__)\n\napp.layout = ht